# Reconhecimento de Imagens com Redes Neurais Convolucionais


## TÓPICOS EM COMPUTAÇÃO IV - DEEP LEARNING e VISÃO COMPUTACIONAL

### Pontifícia Universidade Católica de Minas Gerais

#### Grupo:
* Lauro Milagres Oliveira - 552140
* Lucas Dutra Ponce Donoso de Leon - 553281

## Introdução

Problema:

A densidade da mama é comprovadamente relacionada com o risco do desenvolvimento de câncer, uma vez que mulheres com uma maior densidade mamária podem esconder lesões, levando o câncer a ser detectado tardiamente. A escala de densidade chamada BI-RADS foi desenvolvida pelo American College of Radiology e informa os radiologistas sobre a diminuição da sensibilidade do exame com o aumento da densidade da mama. BI-RADS definem a densidade como sendo quase inteiramente composta por gordura (densidade I), por tecido fibrobroglandular difuso (densidade II), por tecido denso heterogêneo (III) e por tecido extremamente denso (IV). A mamografia é a principal ferramenta de rastreio do câncer e radiologistas avaliam a densidade da mama com base na análise visual das imagens.

Objetivo:

Neste trabalho, você deverá implementar uma rede neural convolucional que receba como entrada uma imagem com resolução de 128x128 pixels e 8 bits por pixel e possibilite o reconhecimento automáticoda densidade da mama.

Solução: Usar uma Rede Neural Convolucional para aprender recursos de imagens e assim prever se uma imagem é do grupo 1, 2, 3 ou 4.

## Definição dos Dados

Total de imagens: 5024, 1.256 para cada classe.

Conjunto de dados de treino: 70% das imagens de densidade BI-RADS.

Conjunto de dados de validação: 70% das imagens de densidade BI-RADS.

Conjunto de dados de teste: 30% do total de imagens ou imagem definida.

## Fonte dos Dados

Usaremos como fonte de dados o que foi disponibilizado pelo Professor ‪Alexei Manso Correa Machado‬.

## Construindo a Rede Neural Convolucional

Para a construção da rede neural, foi utilizado como backend o Keras e TensorFlow, o Keras utiliza o TensorFlow sendo uma biblioteca para simplificar muitos metodos e ajudando na contrução da rede neural.

Segue abaixo os importes usados:

In [1]:
# Imports
import tensorflow as tf
import keras as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from IPython.display import Image
import numpy as np
from keras.preprocessing import image
import os
import random
import shutil

Nós então inicializamos a nossa rede:

In [2]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

Agora nós definimos os parâmetros para o shape dos dados de entrada e a função de ativação. Usaremos 32 features para um array 2D e definiremos nosso array como o formato 3x3.

Converteremos todas as nossas imagens 128x128 pixels em um array de 3 canais de cores.

In [3]:
# Passo 1 - Primeira Camada de Convolução
classifier.add(Conv2D(32, (1, 1),input_shape = (128, 128, 3), activation = 'relu'))

Agora aplicamos o agrupamento (pooling) para reduzir o tamanho do mapa de features resultado da primeira camada de convolução (dividido por 2):

In [4]:
# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Adicionamos então a Segunda Camada de Convolução, tornando nossa rede um pouco mais profunda, e aplicando a camada de pooling à saída da camada de convolução anterior.

In [5]:
# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Adicionamos então a Segunda Camada de Convolução, tornando nossa rede um pouco mais profunda, e aplicando a camada de pooling à saída da camada de convolução anterior.

In [6]:
# Adicionando a Terceira Camada de Convolução
classifier.add(Conv2D(128, (1, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

É aplicado o "achatamento" ou Flatten para converter a estrutura de dados 2D resultado da camada anterior em uma estrutura 1D, ou seja, um vetor.

In [7]:
# Passo 3 - Flattening
classifier.add(Flatten())

No próximo passo conectamos todas as camadas. Usamos uma função de ativação retificadora (relu) e então uma função de ativação sigmóide para obter as probabilidades de cada imagem conter as classes BI-RADS.

In [8]:
# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 4, activation = 'softmax')) #linear

Para compilar a rede, usamos o otimizador "Adam", um excelente algoritmo de primeira ordem para otimização baseada em gradiente de funções objetivas estocásticas, que toma como base uma estimativa adaptada de momentos de baixa ordem.

Usamos uma função log loss com "entropia categorica cruzada", pois ela funciona bem com funções softmax.

Nossa métrica será a acurácia, pois essa é nossa maior preocupação no treinamento deste tipo de modelo.

In [9]:
# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Treinando a Rede Neural Convolucional

### Pré-Processamento

Realizando o pré-processamento nos dados para o treinamento usando a função ImageDataGenerator() do Keras e ajustar escala e zoom das imagens de treino e a escala das imagens de validação.

In [10]:
# Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# train_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)


Aplicamos então os dois objetos criados anteriormente para pré-processar os dados de treino e de validação(que deve ser os mesmos de treino).

Temos também neste bloco, os diretorios usado nesta rede neural, o de traino, validação e teste.

In [11]:
train_dir = '../Topicos/dataset_treino/'
valid_dir = '../Topicos/dataset_validation/'
teste_dir = '../Topicos/dataset_teste/'

'''
Funcao para prepaprar o diretorio, aqui fica definido a regra para divisao das imagens
Sao usados 4 diretorios, que são as classes BI-RADS, deste diretorio principal onde contem as 4 pastas,
são mantidas 70% das imagens e as outras 30% são jogadas em um diretorio de validação.
'''
def load_data(DIR):
    totalImages = 0
    for subDir in os.listdir(DIR):
        tam_dir_train = len(os.listdir(DIR+subDir))
        totalImages = totalImages + int(tam_dir_train*0.30)
        for i in range(int(tam_dir_train*0.30)):
            filename = random.choice(os.listdir(DIR+subDir))
            shutil.move(DIR+subDir+'/'+filename, teste_dir)
            # shutil.copy(DIR+subDir+'/'+filename, teste_dir)
    return int(totalImages/32)


steps_epoch = load_data(train_dir)
print(steps_epoch)

def create_validation_dataset(DIR):
    totalImages = 0
    for subDir in os.listdir(DIR):
        tam_dir_train = len(os.listdir(DIR+subDir))
        for filename in os.listdir(DIR+subDir):
            shutil.copy(DIR+subDir+'/'+filename, valid_dir)

create_validation_dataset(train_dir)

# Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 shuffle=True,
                                                 classes= ['1', '2','3','4'],
                                                 class_mode = 'categorical')

validation_set = validation_datagen.flow_from_directory(valid_dir,
                                                        target_size = (128, 128),
                                                        batch_size = 32,
                                                        shuffle=True,
                                                        class_mode = 'categorical')

47
Found 3520 images belonging to 4 classes.
Found 0 images belonging to 0 classes.


### Treinamento

Usaremos passos definidos pelo total de imagens sobre batch size em nosso conjunto de treinamento para cada época. Escolhemos 2000 etapas de validação para as imagens de validação e 25 epocas.

In [12]:
# Executando o treinamento
classifier.fit(training_set,
                         steps_per_epoch = steps_epoch,
                         epochs = 25,
                         validation_data = validation_set,
                         validation_steps = 2000)

# Salvando o modelo
# classifier.save('../Topicos/model')

Epoch 1/25
47/47 [==============================] - 29s 627ms/step - loss: 1.5210 - accuracy: 0.2367
Epoch 2/25
47/47 [==============================] - 37s 784ms/step - loss: 1.3849 - accuracy: 0.2540
Epoch 3/25
47/47 [==============================] - 27s 570ms/step - loss: 1.3695 - accuracy: 0.2739
Epoch 4/25
47/47 [==============================] - 24s 517ms/step - loss: 1.3040 - accuracy: 0.3730
Epoch 5/25
47/47 [==============================] - 24s 521ms/step - loss: 1.2897 - accuracy: 0.3763
Epoch 6/25
47/47 [==============================] - 25s 525ms/step - loss: 1.2335 - accuracy: 0.4156
Epoch 7/25
47/47 [==============================] - 24s 516ms/step - loss: 1.2249 - accuracy: 0.4249
Epoch 8/25
47/47 [==============================] - 26s 548ms/step - loss: 1.1937 - accuracy: 0.4335
Epoch 9/25
47/47 [==============================] - 25s 529ms/step - loss: 1.2017 - accuracy: 0.4388
Epoch 10/25
47/47 [==============================] - 25s 533ms/step - loss: 1.2152 - accura

## Previsões

Testando o modelo treinado com imagens que ele ainda não viu e que estão nos dados de teste.

Para cada imagem de teste, é carregado as imagens com as mesmas dimensões usadas nas imagens de treino.
Estas imagens por sua vez são convertidas em um array expandindo duas dimensões.

In [13]:
CLASSES = ["BIRADS 1","BIRADS 2","BIRADS 3","BIRADS 4"]

# imagem alearotoria
def getImagemAleatorio(DIR):
    return DIR+random.choice(os.listdir(DIR))

def isCorrect(arq,tipo):
    
    if "_d_" in arq and tipo == "BIRADS 1":
        return "ACERTOU"
    elif "_e_" in arq and tipo == "BIRADS 2":
        return "ACERTOU"
    elif "_f_" in arq and tipo == "BIRADS 3":
        return "ACERTOU"
    elif "_g_" in arq and tipo == "BIRADS 4":
        return "ACERTOU"
    else:
        return "ERROU"

def teste(DIR):
    listTeste = []
    print('==================================================================')
    for file in os.listdir(DIR):
        imagem = DIR+file
        print("Imagem: {}".format(file))
        # Image(filename='dataset_teste/'+file)
        test_image2 = image.load_img(imagem, target_size = (128, 128))
        input_arr = image.img_to_array(test_image2)
        input_arr = np.array([input_arr])  # Convert single image to a batch.
        pred = classifier.predict(input_arr/255)
        birads = np.argmax(pred)
        result = CLASSES[birads]
        
        acertoOuErro = isCorrect(file,result)
        listTeste.append(acertoOuErro)
        
        print("Densidade: {}".format(result))
        #print(acertoOuErro)
        print('==================================================================')
    
    print("Total de imagens de teste: {}".format(len(listTeste)))
    print("Acertou: {} ou {}%".format(listTeste.count('ACERTOU'),round(listTeste.count('ACERTOU')*100/len(listTeste),2)))
    print("Errou: {} ou {}%".format(listTeste.count('ERROU'),round(listTeste.count('ERROU')*100/len(listTeste),2)))

teste(teste_dir)

Imagem: p_d_right_mlo(119).png
Densidade: BIRADS 1
Imagem: p_d_right_mlo(163).png
Densidade: BIRADS 1
Imagem: p_e_right_mlo(214).png
Densidade: BIRADS 4
Imagem: p_d_left_mlo(153).png
Densidade: BIRADS 1
Imagem: p_f_left_mlo(69).png
Densidade: BIRADS 4
Imagem: p_f_right_cc(235).png
Densidade: BIRADS 4
Imagem: p_e_right_cc(217).png
Densidade: BIRADS 3
Imagem: p_e_right_cc(47).png
Densidade: BIRADS 4
Imagem: p_g_left_cc(206).png
Densidade: BIRADS 4
Imagem: p_g_left_mlo(285).png
Densidade: BIRADS 3
Imagem: p_d_right_mlo(107).png
Densidade: BIRADS 2
Imagem: p_f_right_cc(201).png
Densidade: BIRADS 4
Imagem: p_e_left_mlo(59).png
Densidade: BIRADS 2
Imagem: p_f_right_cc(45).png
Densidade: BIRADS 3
Imagem: p_e_left_mlo(25).png
Densidade: BIRADS 2
Imagem: p_d_left_mlo(126).png
Densidade: BIRADS 1
Imagem: p_d_right_cc(309).png
Densidade: BIRADS 1
Imagem: p_g_right_mlo(271).png
Densidade: BIRADS 3
Imagem: p_d_right_mlo(48).png
Densidade: BIRADS 3
Imagem: p_g_right_cc(254).png
Densidade: BIRADS 4
I

O modelo recebe uma imagem que nunca tinha visto antes e com base no que aprendeu durante o treinamento, foi capaz de classificar a imagem como sendo BI-RADS 1, 2, 3 ou 4!

- O modelo treinado nada mais é do que um conjunto de pesos que foram definidos a partir de vetores de imagens de BI-RADS
- Os pesos do modelo foram definidos com base nas características das imagens de treino
- Convertemos a imagem de teste em um vetor de pixels e apresentamos ao modelo
- O modelo compara o vetor da imagem de teste com seus pesos e então emite a classificação

Abaixo, um bloco para rodar com o modelo salvo, ou com imagem definida.

In [7]:
# CLASSES = ["BIRADS 1","BIRADS 2","BIRADS 3","BIRADS 4"]

# loaded_model = tf.keras.models.load_model('../Topicos/model')

# print('##################################################################')
# print('#           Teste com modelo já treinado (Save in Disk)          #')
# print('##################################################################')

# for file in os.listdir(teste_dir):
#     imagem = teste_dir+file
#     print("Imagem: {}".format(imagem))
#     test_image2 = image.load_img(imagem, target_size = (128, 128))
#     input_arr = image.img_to_array(test_image2)
#     input_arr = np.array([input_arr])  # Convert single image to a batch.
#     pred = loaded_model.predict(input_arr/255)
#     birads = np.argmax(pred)
#     print(CLASSES[birads])


# print('##################################################################')
# print('#                       Teste com imagem                         #')
# print('##################################################################')
#imagem = 'dataset_teste/p_e_left_mlo(45).png'
# imagem = 'dataset_teste/alexei1.png'
#imagem = 'dataset_teste/alexei2.png'
# imagem = 'dataset_teste/alexei3.png'
# imagem = 'dataset_teste/alexei4.png'
# test_image = image.load_img(imagem, target_size = (128, 128))
# input_arr = image.img_to_array(test_image)
# input_arr = np.array([input_arr])  # Convert single image to a batch.
# pred = loaded_model.predict(input_arr/255)
# birads = np.argmax(pred)
# print(CLASSES[birads])

##################################################################
#                       Teste com imagem                         #
##################################################################
BIRADS 4


## Conclusão

Ao pesquisar na Internet, vimos que o recomendado para o kernel_size das imagens de entrada com 128x128 pixels seria um tamanho de 1 ou 3. Então optamos por começar com um kernel_size de tamanho 1, passamos para 3 no segundo layer para aprender recursos maiores e retornamos para 1 para reduzir as dimensões espaciais.
Também vimos que era recomendado aumentar a quantidade de filtros nas camadas mais profundas. Assim,  as camadas iniciais na arquitetura de rede (ou seja, mais perto da imagem de entrada real) aprendem menos filtros convolucionais, enquanto as camadas mais profundas na rede (ou seja, mais próximas das previsões de saída) aprenderão mais filtros.
O pool máximo é usado para reduzir as dimensões espaciais do volume de saída de cada camada.